# A Simple Example for EvoGErrors
In this quick demo we want to show the capabilities of differentiating exception types while using **EvoGFuzz**

### Defining a Sample Program
**EvoGErrors** builds its own oracle, which simplifies the process but also adds a level of constraints. In this example, we define an `advanced_calulator` that includes a variety of different error types.

In [1]:
import math

# This function contains various errors that must be found and classified.
def advanced_calculator(inp: str) -> float:

    # The math.increment function does not exist -> AttributeError
    if not str(inp).find("inc") == -1:
        return eval(str(inp), {"inc": math.increment})
    
    if not str(inp).find("factorial") == -1:
        return eval(str(inp))

    if not str(inp).find("is_int") == -1:
        return eval(str(inp))
    
    return eval(
        str(inp), {
            "sqrt": math.sqrt, # Square roots of negative numbers are not defined -> ValueError
            "sin": math.sin,
            "cos": math.cos,
            "tan": math.tan,
            "log": math.log # The logarithm is only defined for positive numbers -> ValueError
            }
    )

def factorial(n):
    if n >= 2:
        return n * factorial(n) # Infinite recursion -> RecursionError
    else:
        return 1

def is_int(inp):
    assert isinstance(inp, int) # If inp is not an integer -> AssertionError
    return True

### Defining the Grammar
We need to define a corresponding grammar. This grammar covers all potentially valid inputs to the calculator function, including mathematical expressions like square roots, trigonometric functions, factorial, division, incrementing and length of a number, as well as checking whether a number is an integer. Also, the grammar includes one common mistake, which is later correctly found by **EvoGErrors**. Perhaps you already see it.

In [2]:
from fuzzingbook.Grammars import Grammar, is_valid_grammar

CALCGRAMMAR: Grammar = {
    "<start>":
        ["<start1>"],

    "<start1>":
        ["<expr>", "<term> / <term>"],

    "<expr>":
        ["<function>(<term>)"],

    "<function>":
        ["sqrt", "tan", "cos", "sin", "log", "len", "inc", "factorial", "is_int"],
    
    "<term>":
        ["-<value>", "<value>"], 
    
    "<value>":
        ["<integer>.<integer>", "<integer>"],

    "<integer>":
        ["<digit><integer>", "<digit>"],

    "<digit>":
        ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0"]
}

assert is_valid_grammar(CALCGRAMMAR)

### Initial Inputs
Initial inputs can be defined but it is recommended not to, since **EvoGErrors** needs to check for every error type. Unfortunate inputs would increase the run time immensely.

In [3]:
from evogfuzz.evogerrors import EvoGErrors

epp = EvoGErrors(
    grammar=CALCGRAMMAR,
    program=advanced_calculator,
    min_iterations = 3,
    final_iterations = 30
)

### Iterations

Note that **EvoGErrors** in contrast to **EvoGFuzz** takes two different iteration parameters. The `min_iterations` is executed for every exception type, it is advisable to keep the number low. Once **EvoGErrors** found a bug, it continues the iteration until it hits the specified `final_iterations`.

### Let's Start the Fuzzing

It may take a while. Try restarting if it takes too long.

In [4]:
found_exception_inputs = epp.fuzz()

[1/65] Currently searching for: ArithmeticError
[2/65] Currently searching for: AssertionError
[2/65] Looking deeper into: AssertionError
[3/65] Currently searching for: AttributeError
[3/65] Looking deeper into: AttributeError
[4/65] Currently searching for: BaseException
[5/65] Currently searching for: BlockingIOError
[6/65] Currently searching for: BrokenPipeError
[7/65] Currently searching for: BufferError
[8/65] Currently searching for: BytesWarning
[9/65] Currently searching for: ChildProcessError
[10/65] Currently searching for: ConnectionAbortedError
[11/65] Currently searching for: ConnectionError
[12/65] Currently searching for: ConnectionRefusedError
[13/65] Currently searching for: ConnectionResetError
[14/65] Currently searching for: DeprecationWarning
[15/65] Currently searching for: EOFError
[16/65] Currently searching for: EncodingWarning
[17/65] Currently searching for: OSError
[18/65] Currently searching for: Exception
[19/65] Currently searching for: FileExistsError


In [5]:
print(f"EvoGFuzz found {len(found_exception_inputs)} Exception types!")
for type, inps in found_exception_inputs.items():
    print(type.__name__)

EvoGFuzz found 7 Exception types!
AssertionError
AttributeError
RecursionError
SyntaxError
TypeError
ValueError
ZeroDivisionError


If you were lucky, you found all 7 Exceptions. The `SyntaxError` is one that wasn't coming from the calculator. Let's see what sample inputs are for each error type.

In [6]:
for type, inputs in found_exception_inputs.items():
    print(f"{type.__name__}:")
    print()
    for input in list(inputs)[:3]:
        print(str(input))
    print()

AssertionError:

is_int(-4.246)
is_int(-44.2954)
is_int(-29.803520)

AttributeError:

inc(-99032397779987807233)
inc(-37.54775567398668)
inc(9.511)

RecursionError:

factorial(8.3)
factorial(75.877)
factorial(43454581)

SyntaxError:

-305535030 / -0150
09 / 0
8 / -043

TypeError:

len(2.74)
len(398.19)
len(4.688)

ValueError:

log(-19459423582168871241929917)
sqrt(-9.6321)
log(-6.8)

ZeroDivisionError:

8903.0240794 / 0
-5 / 0
89 / 00



The `SyntaxError` was introduced by the common mistake to not check for leading zeros in the grammar.

### Multi Probabilistic Grammar

Lastly, let's visualize the probabilities by combining all probabilistic grammars into one. 

In [7]:
print(epp.get_last_grammar())

<start>:
   ('<start1>', {'AssertionError': None, 'AttributeError': None, 'RecursionError': None, 'SyntaxError': None, 'TypeError': None, 'ValueError': None, 'ZeroDivisionError': None})
<start1>:
   ('<expr>', {'AssertionError': 0.4846, 'AttributeError': 1.0, 'RecursionError': 0.76, 'SyntaxError': 0.68, 'TypeError': 1.0, 'ValueError': 0.96, 'ZeroDivisionError': 0.9444})
 | ('<term> / <term>', {'AssertionError': 0.5154, 'AttributeError': 0.0, 'RecursionError': 0.24, 'SyntaxError': 0.32, 'TypeError': 0.0, 'ValueError': 0.04, 'ZeroDivisionError': 0.0556})
<expr>:
   ('<function>(<term>)', {'AssertionError': None, 'AttributeError': None, 'RecursionError': None, 'SyntaxError': None, 'TypeError': None, 'ValueError': None, 'ZeroDivisionError': None})
<function>:
   ('sqrt', {'AssertionError': 0.0, 'AttributeError': 0.0, 'RecursionError': 0.0, 'SyntaxError': 0.0, 'TypeError': 0.0396, 'ValueError': 0.0, 'ZeroDivisionError': 0.0})
 | ('tan', {'AssertionError': 0.0, 'AttributeError': 0.0, 'Recurs